Osnabrück University - Machine Learning (Summer Term 2016) - Prof. Dr.-Ing. G. Heidemann, Ulf Krumnack

# Exercise Sheet 02: Decision Trees

## Assignment 1: Decision Trees [2 Points]

Draw the decision trees for the following boolean functions. Either use pen and paper and bring the results to the feedback session or employ your ASCII artist within below. 

Note: $\oplus := xor$, that means one of the operands has to be true, while the other one has to be false:

$\oplus$ | $B$ | $\neg B$
---------|-----|---------
$A$      |  f  |    t
$\neg A$ |  t  |    f

### a) $\neg A \wedge B$

### b) $A \oplus B$ 

### c) $A \vee (B \wedge C) \vee (\neg C \wedge D)$

### d) $(A \rightarrow (B \wedge \neg C)) \vee (A \wedge B)$

$= (\neg A \vee (B \wedge \neg C)) \vee (A \wedge B)$

$= \neg A \vee (A \wedge B) \vee (B \wedge \neg C)$

$= \neg A \vee (A \wedge B)$

## Assignment 2: Entropy and Information Gain [10 Points]

In many machine learning applications it is crucial to determine which criterions are necessary for a good classification. Decision trees have those criterions close to the root, imposing an order from significant to less significant criterions. One way to select the most important criterion is to compare its information gain or its entropy to others. The following dataset is a hands-on example for this method. 

Consider the following attributes with their possible values:

  * $raining = \{yes, no\}$
  * $tired = \{yes, no\}$
  * $late = \{yes, no\}$
  * $distance = \{short, medium, long\}$

And a training data set consisting of those attributes:

| #  | raining | tired | late | distance | attend_party |
|----|---------|-------|------|----------|--------------|
| 1  | yes     | no    | no   | short    | **yes**      |
| 2  | yes     | no    | yes  | medium   | **no**       |
| 3  | no      | yes   | no   | long     | **no**       |
| 4  | yes     | yes   | yes  | short    | **no**       |
| 5  | yes     | no    | no   | short    | **yes**      |
| 6  | no      | no    | no   | medium   | **yes**      |
| 7  | no      | yes   | no   | long     | **no**       |
| 8  | yes     | no    | yes  | short    | **no**       |
| 9  | yes     | yes   | no   | short    | **yes**      |
| 10 | no      | yes   | no   | medium   | **no**       |
| 11 | no      | yes   | no   | long     | **no**       |
| 12 | no      | yes   | yes  | short    | **no**       |

### a)

Build the root node of a decision tree from the training samples given in the table above by calculating the information gain for all four attributes (raining, tired, late, distance).

$$Gain(S,A) = Entropy(S) - \sum_{v \in Values(A)} \frac{|S_v|}{|S|}Entropy(S_v)$$

$$Entropy(S) = -p_{\oplus} log_{2} p_{\oplus} - p_{\ominus} log_{2} p_{\ominus}$$

$S$ is the set of all data samples. $S_v$ is the subset for which attribute $A$ has value $v$. An example for attribute **tired** with value $yes$ would be:
$$|S_{yes}| = 7, S_{yes}:[1+, 6−]$$

### b)

Perform the same calculation as in **a)** but use the gain ratio instead of the information gain. Does the result for the root node change?

$$GainRatio(S,A) = \frac{Gain(S,A)}{SplitInformation(S,A)}$$

$$SplitInformation(S,A) = - \sum_{v \in Values(A)} \frac{|S_v|}{|S|} log_{2} \frac{S_{v}}{S}$$

## Assignment 3: ID3 algorithm [5 Points]

Implement the following two functions in Python. Take a look at the `assert`s to see how the function should behave. An assert is a condition that your function is required to pass. Most of the conditions here are taken from the lecture slides (ML-03, Slide 12 & 13). Don't worry if you do not get all asserts to pass, just comment those out.

### a) Entropy

$$Entropy(S) = - \sum_{i=1...c} p_i log_2 p_i$$

In [ ]:
from math import log
def entropy(S):
    """
    Calculate the entropy for a given target value set. 
    S: List of target classes for specific observations.
    """
    p_i = [len([value for value in S if value == c])/len(S) for c in set(S)]
    return - sum(p * log(p, 2) for p in p_i) 


# See ML-03, Slide 12 & 13
assert entropy([1,1,1,0,0,0]) == 1.0
assert round(entropy([1,1,1,1,0,0,0]), 3) == 0.985
assert round(entropy([1,1,1,1,1,1,0]), 3) == 0.592
assert round(entropy([1,1,1,1,1,1,0,0]), 3) == 0.811
assert round(entropy([2,2,1,1,0,0]), 3) == 1.585
assert round(entropy([2,2,2,1,0]), 3) == 1.371
assert round(entropy([2,2,2,0,0]), 3) == 0.971

### b)  Information Gain

$$Gain(S,A) = Entropy(S) - \sum_{v \in Values(A)} \frac{|S_v|}{|S|} Entropy(S_v)$$

In [ ]:
def gain(S, A):
    """
    Calculates the expected reduction in entropy due to sorting on A.
    S: Target classes for observations in A.
    A: Observations.
    """
    sigma = 0
    for v in set(A): # sets only contain unique values
        S_v = [S[key] for (key, v_) in enumerate(A) if v_ == v]
        sigma = sigma + ((len(S_v) / len(S)) * entropy(S_v))
    return entropy(S) - sigma


# See ML-03, Slide 12 & 13
assert_S_ = [0,0,1,1,1,0,1,0,1,1,1,1,1,0]
assert round(gain(assert_S_, [1,1,1,1,0,0,0,1,0,0,0,1,0,1]), 3) == .152
assert round(gain(assert_S_, [0,1,0,0,0,1,1,0,0,0,1,1,0,1]), 3) == .048

### c) ID3

In the next cell we have implemented the ID3 algorithm after the pseudocode on [Wikipedia](https://en.wikipedia.org/wiki/ID3_algorithm#Pseudocode). You may look through the code but it is not necessary for the exercise (you just have to run it). In the following cell the algorithm is applied on two datasets. What is the difference? For which dataset is the ID3 algorithm better suited and why?

In [ ]:
from collections import namedtuple

class Node(namedtuple('Node', 'label children')):
    """
    A small node representation with a pretty string representation.
    """
    def __str__(self, level=0):
        return_str ='{}{!s}\n'.format(' ' * level * 4, self.label)
        for child in self.children:
            return_str += child.__str__(level + 1)
        return return_str

In [ ]:
from collections import Counter

def id3(examples, attributes, target_attribute = None): 
    """
    Calculate a tree of Nodes (fields: label [string], children [list]) 
    using the ID3 algorithm found as pseudocode on Wikipedia. Including comments (currently).
    """
    # Create a root node for the tree
    # If all examples are positive, Return the single-node tree Root, with label = 
    # If all examples are negative, Return the single-node tree Root, with label = -.
    if all(target == examples['targets'][0] for target in examples['targets']):
        return Node('Result: {!s}'.format(examples['target_names'][examples['targets'][0]]), [])
    
    # If number of predicting attributes is empty, then Return the single node tree Root,
    # with label = most common value of the target attribute in the examples.
    if len(attributes) == 0:
        attr = Counter(data_sample[target_attribute] for data_sample in examples['data']).most_common(1)
        return Node('Attribute: {!s}, {!s} occurences'.format(examples['attributes'][target_attribute], attr), [])
    
    # A <-- The Attribute that best classifies examples.
    gains = [gain(examples['targets'], [r[attribute] for r in examples['data']]) for attribute in attributes]
    attribute = attributes[gains.index(min(gains))]

    # Create a root node for the tree (2)
    # Decision Tree attribute for Root = A.
    root = Node('Attribute: {!s} (gain {!s})'.format(examples['attributes'][attribute], round(min(gains), 4)), [])
    
    # Otherwise Begin
    # For each possible value, vi, of A,
    for vi in set(data_sample[attribute] for data_sample in examples['data']):
        # Add a new tree branch below Root, corresponding to the test A = vi.
        child = Node('Value: {!s}'.format(vi), [])
        root.children.append(child)
        # Let Examples(vi) be the subset of examples that have the value vi for A
        vi_indices = [idx for idx, data_sample in enumerate(examples['data']) if data_sample[attribute] == vi]
        examples_vi = dict(examples)
        examples_vi['data'] = [examples['data'][i] for i in vi_indices]
        examples_vi['targets'] = [examples['targets'][i] for i in vi_indices]
        # If Examples(vi) is empty
        if len(examples_vi['data']) == 0:
            # Then below this new branch add a leaf node with label = most common target value in the examples
            attr = Counter(examples_vi['targets']).most_common(1)
            label = 'Attribute: {!s}, {!s} occurences'.format(examples['attributes'][target_attribute], attr)
            child.children.append(Node(label, []))
        else:
            # Else below this new branch add the subtree ID3 (Examples(vi), Attributes – {A})
            child.children.append(
                id3(examples_vi, \
                    [attribute_ for attribute_ in attributes if not attribute_ == attribute], \
                    attribute)
            )
    # End

    # Return root
    return root

This code runs the ID3 algorithm on the party data set which you already know from Assignment 2.

In [ ]:
import json

with open('party.json', 'r') as party_file:
    party = json.load(party_file)

# Make sure our gain function handles the data set as expected.
assert round(gain(party['targets'], [r[2] for r in party['data']]), 3) == 0.252

# Apply ID3 algorithm
tree_party = id3(party, list(range(len(party['attributes']))))

print(tree_party)

This code runs the ID3 algorithm on the famous iris flowser data set. Have a look at [Wikipedia](https://en.wikipedia.org/wiki/Iris_flower_data_set) if you want more infos about it.

In [ ]:
import json

with open('iris.json', 'r') as iris_file:
    iris = json.load(iris_file)

# Make sure our gain function handles the data set as expected.
assert round(gain(iris['targets'], [r[2] for r in iris['data']]), 3) == 1.446

# Apply ID3 algorithm
tree_iris = id3(iris, list(range(len(iris['attributes']))))

print(tree_iris)

## Assignment 4: Decision Tree on Ionosphere Dataset [5 Points]

In this exercise we are going to examine and compare two Decision Trees that were generated from the Iris flower data set ([Wiki](https://en.wikipedia.org/wiki/Iris_flower_data_set)) where three variations of the iris flower are quantified. The Iris data set is a classical example of a labeled dataset, i.e. every sample consists of two parts: features and labels. There are 4 features per sample in this dataset (sepal length, sepal width, petal length and petal width in cm) and a corresponding label (Iris Setosa, Iris Versicolour, Iris Virginica). These samples are by nature **noisy**, no matter how carefully the measurement was taken - slight deviation from the actual length **cannot be avoided**. We want to learn how the features are related to the label so that we could (in the future) predict the label of a new sample automatically. One way to obtain such a `classifier` is to train a decision tree on the data.

The following two trees were generated from the same dataset with matlab's `fitctree()` function.
### Tree 1:

### Tree 2:

### a)
What does it mean that features $x1$ and $x2$ do not appear at any of the nodes of the decision tree?

*Sepal length and sepal width are not relevant for the classification. This might be either because they are redundant or because they are independent of the class.*

### b)
With which method from the lecture might the second tree have been generated from the first one? Explain the procedure.

*Reduced error pruning. Greedily remove the node that reduces error on validation set the most.*

### c)

After training the tree we can calculate the accuracy, i.e. the percentage of the training set that is classified correctly. Although the first tree was trained on the dataset until no improvement of the accuracy was possible, its accuracy is *only* 98%. Explain why it is not 100 %

*The dataset is probably inconsistent, i.e. there are samples with the same features but different classes. Alternative: the thresholding has not produced the optimal partitioning.*

### d)
Tree number 2 only has a 96% accuracy on the training set. Why might this tree still be preferable over tree 1?

*Tree 1 probably overfitted the training data, i.e. it has not only captured the structure but also the noise in the data. It probably won't generalize as well as the second tree.
Another advantage of tree 2 is that it is faster at classifiying new data since less computations have to be made. This difference is hardly noticible however.*